This notebook contains code designed for data inspection and filtering, with the goal of creating classification datasets suitable for machine learning predictive tasks. Specifically, we replicate the steps used for the 2022 database, with filters numbers following it. This enables us to use the resulting datasets as validation sets for models trained with data from the main datasets. For comprehensive details about these datasets, including information about data collection methods, sources, and a dictionary of variables, please refer to the project repository on GitHub: \url{https://github.com/gabivaleriano/HealthDataBR}

In [1]:
library(tidyverse)

In [2]:
data <- read.csv("CHIKBR20.csv")

### Filter 1: remove duplicates

In [ ]:
# check and remove duplicates

data %>% dim
data %>% unique %>% dim
data %>% unique -> data

In [ ]:
# inspecting data distribution according to state

data %>% count(SG_UF_NOT)

In [ ]:
# visualize data

data %>% head
data %>% colnames

In [ ]:
# since ano-nasc is not present is this database, age will be computed from nu_idade_n

data %>% count(NU_IDADE_N) %>% head

In [ ]:
# the information must contain at least 4 digits
# when it starts with 10 - hours, 20 - days, 30 - months, 40 - years 

# remove information with less than 4 digits

data %>% filter(NU_IDADE_N > 1000) %>% count(NU_IDADE_N) %>% head

data %>% filter(NU_IDADE_N > 1000) -> data



In [ ]:
# remove information that is not coherent

data %>% filter(!(NU_IDADE_N > 2030 & NU_IDADE_N < 3000)) %>% filter(NU_IDADE_N> 2025) %>% 
    count(NU_IDADE_N) %>% head

data %>% filter(!(NU_IDADE_N > 2030 & NU_IDADE_N < 3000)) -> data


In [ ]:
# remove information that is not coherent

data %>% filter(!(NU_IDADE_N > 3012 & NU_IDADE_N < 4000)) %>% filter(NU_IDADE_N> 3010) %>% 
    count(NU_IDADE_N) %>% head

data %>% filter(!(NU_IDADE_N > 3012 & NU_IDADE_N < 4000)) ->data


In [ ]:
# transform in years 

# if is less than 4000 the patient has not complete one year
data %>% mutate(age = ifelse(NU_IDADE_N < 4000, 0, (NU_IDADE_N - round(NU_IDADE_N, -2)))) %>% 
    select(age, NU_IDADE_N) %>% head

data %>% mutate(age = ifelse(NU_IDADE_N < 4000, 0, (NU_IDADE_N - 4000))) -> data

In [ ]:
# inspect possible target features

data %>% count(EVOLUCAO)
data %>% count(HOSPITALIZ)

data %>% dim

In [ ]:
# check how many regions, states and cities are present in the database

data %>% select(ID_REGIONA) %>% unique %>% dim
data %>% select(ID_MUNICIP) %>% unique %>% dim
data %>% select(SG_UF_NOT) %>% unique %>% dim
data %>% select(SG_UF) %>% unique %>% dim

data %>% select(ID_MN_RESI) %>% unique %>% dim
data %>% select(ID_PAIS) %>% unique %>% dim
data %>% select(ID_RG_RESI) %>% unique %>% dim

data %>% select(ID_UNIDADE) %>% unique %>% dim

data %>% select(UF) %>% unique %>% dim
data %>% select(MUNICIPIO) %>% unique %>% dim

### Delete columns

In [ ]:
#keep only id related with the city

data %>% select(-c(ID_REGIONA, ID_MUNICIP, SG_UF, ID_PAIS, ID_RG_RESI, UF, MUNICIPIO)) -> data

# mostly NA values

data %>% count(TPAUTOCTO)
data %>% count(COPAISINF)
data %>% count(COMUNINF) 
data %>% count(TP_SISTEMA)

data %>% count(NDUPLIC_N)
data %>% count(COMPLICA)
data %>% count(CON_FHD)

data %>% count(SOROTIPO)
data %>% count(HISTOPA_N)
data %>% count(IMUNOH_N)
data %>% count(DOENCA_TRA)
data %>% count(ALRM_HIPOT)

data %>% count(TP_NOT)
data %>% count(ID_AGRAVO)
data %>% count(NU_ANO)


data %>% select(-c(TP_NOT, ID_AGRAVO, NU_ANO)) -> data
data %>% select(-c(TPAUTOCTO, COPAISINF, COUFINF, COMUNINF)) -> data
data %>% select(-c(TP_SISTEMA)) -> data
data %>% select(-c(NDUPLIC_N, COMPLICA, CON_FHD)) -> data
data %>% select(-c(SOROTIPO, HISTOPA_N, IMUNOH_N, DOENCA_TRA, ALRM_HIPOT)) -> data
data %>% select(-c(NU_IDADE_N)) -> data

# this dates usually are the same and refer to the date of digitalization

data %>% select(-c(DT_NOTIFIC, DT_ENCERRA, DT_INVEST, CRITERIO)) -> data

data %>% select(ALRM_PLAQ:ALRM_LIQ) %>% summary
data %>% select(GRAV_PULSO:GRAV_ORGAO) %>% summary
data %>% select(MANI_HEMOR:PLAQ_MENOR) %>% summary

data %>% select(-c(ALRM_PLAQ:PLAQ_MENOR)) -> data 

data %>% dim

data %>% colnames

### Filter 2: Remove if is na for state or health unit

In [ ]:
data %>% filter(!is.na(SG_UF_NOT)) -> data

data %>% dim

data %>% count(EVOLUCAO)
data %>% count(HOSPITALIZ)

### Remove columns 

In [ ]:
# ID_OCUPA_N seems to not be filled consistently
# it refers to area of activity and should be filled with the same value (not aplicable) to children 
# sem_not is the epidemiologic week of notification, sem_pri is the epidemiologic week of the first symptoms

data %>% filter(age< 5) %>% count(ID_OCUPA_N) %>% slice(1:15)
data %>% select(-c(ID_OCUPA_N, SEM_NOT)) -> data

### Filter 4: remove patients older than 110 years and younger than 12

In [ ]:
# remove patients older than 110 years old 
data %>% filter(age < 111) -> data

# remove patients younger than 16 years old
data %>% filter(age > 11) -> data

data %>% dim

data %>% count(EVOLUCAO)
data %>% count(HOSPITALIZ)

### Filter 5: remove patients without sex information

In [ ]:
# remove patients without sex information

data %>% count(CS_SEXO)
data %>% filter(CS_SEXO == 'F' | CS_SEXO == 'M') %>% count(CS_SEXO)
data %>% filter(CS_SEXO == 'F' | CS_SEXO == 'M') -> data
data %>% dim

# create a new column with numeric values

data %>% mutate(sex = if_else(CS_SEXO == 'M', 0, 1)) %>% count(sex)
data %>% mutate(sex = if_else(CS_SEXO == 'M', 0, 1)) -> data

# remove the former column
data %>% select(-CS_SEXO) -> data

data %>% dim

data %>% count(EVOLUCAO)
data %>% count(HOSPITALIZ)

In [ ]:
# check the column pregnancy for each sex
# transform na values in the column pregnancy in 0 (not pregnant)

data %>% count(CS_GESTANT)

data %>% filter(sex == 1) %>% count(CS_GESTANT)

data %>% filter(sex == 0) %>% count(CS_GESTANT)

data %>% mutate(CS_GESTANT = if_else(is.na(CS_GESTANT), 0, CS_GESTANT)) %>% count(CS_GESTANT)

data %>% mutate(CS_GESTANT = if_else(is.na(CS_GESTANT), 0, CS_GESTANT)) -> data

In [ ]:
# transform the column pregnant in binary, orignally diferent values refer to difernt stages on pregnancy

data %>% mutate(pregnant = if_else((CS_GESTANT == 1 | 
                               CS_GESTANT == 2 | 
                               CS_GESTANT == 3 | 
                               CS_GESTANT == 4), 1, 0)) %>% count(pregnant)


data %>% mutate(pregnant = if_else((CS_GESTANT == 1 | 
                               CS_GESTANT == 2 | 
                               CS_GESTANT == 3 | 
                               CS_GESTANT == 4), 1, 0)) -> data

data %>% select(-CS_GESTANT) -> data

### Filter 6: remove patients without race information

In [ ]:
# remove unknown race, or na values

data %>% count(CS_RACA)

data %>% filter(!is.na(CS_RACA)) %>% filter(CS_RACA < 9) %>% count(CS_RACA)

data %>% filter(!is.na(CS_RACA)) %>% filter(CS_RACA < 9) -> data

data %>% dim

data %>% count(EVOLUCAO)
data %>% count(HOSPITALIZ)

In [ ]:
# check year of schooling. 9 = unknown. 
data %>% count(CS_ESCOL_N)

data$CS_ESCOL_N[data$CS_ESCOL_N == 9] <- NA

data %>% count(CS_ESCOL_N)

In [ ]:
# how many groups by age, race, city and sex
data %>% group_by(age, CS_RACA, ID_MN_RESI, sex) %>% n_groups

# create a new column with the average
data %>% group_by(age, CS_RACA, ID_MN_RESI, sex) %>% mutate(average = mean(CS_ESCOL_N, na.rm=TRUE)) %>% 
    ungroup -> data

# when CS_ESCOL_N is na change by the average
data %>% mutate(CS_ESCOL_N = if_else((is.na(CS_ESCOL_N)), round(average), CS_ESCOL_N)) %>% count(CS_ESCOL_N)
data %>% mutate(CS_ESCOL_N = if_else((is.na(CS_ESCOL_N)), round(average), CS_ESCOL_N)) -> data
data %>% select(-average) -> data

In [ ]:
# create a new column with the average
data %>% group_by(age, CS_RACA, sex) %>% mutate(average = mean(CS_ESCOL_N, na.rm=TRUE)) %>% 
    ungroup -> data

# when CS_ESCOL_N is na change by the average
data %>% mutate(CS_ESCOL_N = if_else((is.na(CS_ESCOL_N)), round(average), CS_ESCOL_N)) %>% count(CS_ESCOL_N)
data %>% mutate(CS_ESCOL_N = if_else((is.na(CS_ESCOL_N)), round(average), CS_ESCOL_N)) -> data
data %>% select(-average) -> data

### Filter 7: remove patients without schooling information (after input by group)

In [ ]:
# if is still na, drop it

data %>% count(CS_ESCOL_N)

data %>% filter(!(is.na(CS_ESCOL_N))) -> data

data %>% count(EVOLUCAO)
data %>% count(HOSPITALIZ)

data %>% dim

### Delete columns 

In [ ]:
# this feature is correlated with uf 

data %>% select(-ID_MN_RESI) -> data

In [ ]:
# check the presence of na's inside the columns of symptoms and comorbities

sum(is.na(data %>% select(FEBRE:AUTO_IMUNE)))

In [ ]:
# check if the NA's are in the same rows

data %>% filter(!is.na(FEBRE)) -> teste
sum(is.na(teste %>% select(FEBRE:AUTO_IMUNE)))

### Filter: rows with NA for the symptoms

In [ ]:
# filter rows with NA for the symptoms

data %>% filter(!is.na(FEBRE)) -> data

data %>% dim
data %>% count(EVOLUCAO)
data %>% count(HOSPITALIZ)

In [ ]:
# the value 2 means the non-occurrence of a symptom, change this to 0

data %>% select(FEBRE:AUTO_IMUNE, HOSPITALIZ) -> symptoms
data %>% select(!(FEBRE:AUTO_IMUNE)) %>% select(-c(HOSPITALIZ)) -> others

symptoms %>% head

symptoms <- symptoms %>%
  mutate_all(~ ifelse(. == 2, 0, .))

symptoms %>% head

symptoms %>% cbind(others) -> data

data$EVOLUCAO[data$EVOLUCAO == 1] <- 0
data$EVOLUCAO[data$EVOLUCAO == 2] <- 1

### Filter 8: keep only patients in the acute stage of the disease

In [ ]:
# only patients in the acute stage 

data %>% filter(CLINC_CHIK == 1) -> data
data %>% select(-CLINC_CHIK) -> data

In [ ]:
# date of the first symptoms 

# change format

data$DT_SIN_PRI <- as.Date(data$DT_SIN_PRI)

# first and last date

data %>% select(DT_SIN_PRI) %>% arrange(DT_SIN_PRI) %>% slice(1)
data %>% select(DT_SIN_PRI) %>% arrange(desc(DT_SIN_PRI)) %>% slice(1)



### Filter: remove notifications from previous years

In [ ]:
data %>% filter(SEM_PRI > 202000) -> data

# change date format

data %>% mutate(SEM_PRI = SEM_PRI - 202200) -> data

data %>% dim
data %>% colnames

In [ ]:
data %>% select(-c('DT_CHIK_S1','DT_CHIK_S2','DT_PRNT',
                   'RES_CHIKS1','RES_CHIKS2','RESUL_PRNT',
                   'DT_SORO','RESUL_SORO','DT_NS1','RESUL_NS1','DT_VIRAL',
                   'RESUL_VI_N','DT_PCR','RESUL_PCR_')) -> data

In [ ]:
colnames(data) <- c("fever", 'myalgia','headache',
                  'exanthema', 'vomiting','nausea',
                  'back_pain','conjunctivitis', 'arthritis', 
                  'arthralgia', 'petechiae', 'leukopenia', 
                  'lasso_prove', 'retro_orbital_pain', 'diabetes', 
                  'hematological_diseases', 'hepatopathies', 'chronic_kidney_disease', 
                  'arterial_hypertension', 'acid_peptic_disease', 'autoimmune_diseases',
                  'hospitalization','id_state','id_place',
                  'dt_first_symptoms', 'epidemiological_week', 
                  'race', 'schooling_years', 'dt_hospitalization', 
                  'chikungunya', 'death', 'dt_death', 
                  'age', 'sex', 'pregnancy')

### Filter 9: only patients cured or dead with the disease 

1- cura (cured), 2- óbito pelo agravo (death by chikungunya) 3- óbito por outras causas (death by other reasons) 4- óbito em investigação (death under investigation) 9- ignorado (ignored)

In [ ]:
# only cured or dead by the disease

data %>% count(death)
data %>% filter(death == 1 | death == 0) -> data

data %>% dim

data %>% count(death)

In [ ]:
# filter hospitalized patients

data %>% filter(hospitalization == 1) %>% count(death)
data %>% filter(hospitalization == 1) -> data

In [ ]:
# create a column with the number of days between the disease investigation and death

data$dt_hospitalization <- as.Date(data$dt_hospitalization)
data$dt_death <- as.Date(data$dt_death)

data %>% mutate(days = difftime(dt_death, dt_hospitalization, units = 'days')) -> data

data$days <- as.numeric(data$days)

data %>% count(days)

### Filter 10: remove patients that died with more than 30 days after first symptoms

In [ ]:
# remove patients that died after 30 days of the first sympthoms

data %>% filter(days < 31| is.na(days)) -> data

data %>% count(death)

In [ ]:
data %>% select(c('fever','myalgia','headache','exanthema','vomiting','nausea','back_pain','arthritis',
                  'arthralgia','diabetes','chronic_kidney_disease','arterial_hypertension','id_state',
                  'id_place','epidemiological_week','race','schooling_years','death','age','sex')) -> data

In [ ]:
data %>% colnames

data %>% dim

data %>% count(death)

data %>% select(fever:schooling_years, age, sex, death) -> data
data %>% write_csv('chikungunya_death_dataset_2020.csv')